# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.txt` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [57]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, category, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `category` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talk_md_files/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [58]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [59]:
talks = pd.read_csv("talks.txt", sep="\t", header=0, encoding = "cp932")
talks

,title,category,url_slug,venue,date,location,talk_url,description
0,Theoretical Study of the Photo-Induced Magneti...,poster,JKT22,The 22nd Japan-Korea-Taiwan Symposium on Stron...,March 15 2024,"Kagoshima University, Japan",https://okuda812.wixsite.com/jkt22,NaN
1,Theoretical Study of the Photo-Induced Magneti...,poster,Tanabe-Sugano70,"70 years of the Tanabe-Sugano diagrams,",September 10 2024,"Shinjuku, Japan",https://sites.google.com/view/70years-of-tanab...,NaN
2,立方格子上の近藤格子模型における光誘起磁気転移の理論,oral,JPS2024fall,日本物理学会 第79回年次大会（2024年）,September 16 2024,"Hokkaido University, Japan",https://onsite.gakkai-web.net/jps/jps_search/2...,NaN
3,3次元近藤格子模型における光誘起磁気相転移の微視的機構の理論,oral,JPS2025spring,日本物理学会 2025年春季大会,March 18 2025,Online,https://onsite.gakkai-web.net/jps/jps_search/2...,NaN
4,立方格子上の近藤格子模型における光誘起磁気転移の理論,poster,summerschool2025,第70回物性若手夏の学校,August 2 2025,"Shiga, Japan",https://cmpss.jp/,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [60]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each category. It does the YAML metadata first, then does the description for the individual page.

In [61]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = item.url_slug + ".md"
    html_filename = item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talk_md_files" + "\n"
    
    if len(str(item.category)) > 3:
        md += 'category: "' + item.category + '"\n'
    else:
        md += 'category: "Talk"\n'
    
    md += "permalink: /talk_md_files/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talk_md_files/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [62]:
!ls ../_talk_md_files

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [63]:
!cat ../_talk_md_files/2013-03-01-tutorial-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
